
# Проект‑практикум: Парсинг данных • Вариант для студентов

**Задача:** реализовать рабочий пайплайн парсинга (сбор → очистка → проверка) своими руками.  
**Правило:** в кодовых ячейках ниже оставлены только комментарии‑подсказки. Заменяйте их на **свой код**.

**Этапы**
1. Фаза 0 — подготовка окружения и структуры.
2. Фаза 1 — мини‑парсер «Quotes to Scrape».
3. Фаза 2 — маркетплейс (двухшаговый: категория → карточка).
4. Фаза 3 — очистка и валидация.
5. Фаза 4 — воспроизводимость (мини‑конвейер) и смоук‑тесты.

> Подсказка: используйте понятные имена переменных, функции с докстрингами и логирование.


## 1. **Фаза 0 — подготовка окружения и структуры**

Это как настройка рабочего стола перед началом проекта:

* мы устанавливаем все нужные программы и библиотеки (инструменты),
* создаём папки для хранения данных: `raw` (сырые), `processed` (очищенные), `logs` (логи).

👉 Цель: подготовить «чистое рабочее место», чтобы дальше всё было организованно.

---

### 2. **Фаза 1 — мини-парсер «Quotes to Scrape»**

Это тренировочный этап. Мы берём сайт для обучения (quotes.toscrape.com), где размещены цитаты.

* Скачиваем их при помощи кода на Python,
* сохраняем в таблицу (CSV) с колонками «цитата», «автор», «теги».

👉 Цель: научиться базовому парсингу и проверить, что связка `requests + BeautifulSoup + CSV` работает.

---

### 3. **Фаза 2 — маркетплейс (категория → карточка)**

Здесь уже настоящий сайт — маркетплейс (Lalafo).

* Находим страницу категории (например, электроника),
* собираем ссылки на карточки товаров,
* переходим в каждую карточку и вытаскиваем данные: название товара и цену.

👉 Цель: освоить более сложный сценарий «двухшагового» парсинга: сначала ссылки, потом сами объекты.

---

### 4. **Фаза 3 — очистка и валидация**

Сырые данные редко бывают аккуратными. Поэтому:

* удаляем дубликаты и пустые строки,
* приводим цены к числам и фильтруем нереалистичные значения,
* добавляем служебные поля (источник, дата сбора, категория).

👉 Цель: получить чистый датасет, готовый для анализа или загрузки в базу.

---

### 5. **Фаза 4 — воспроизводимость (пайплайн) и смоук-тесты**

Здесь мы собираем всё вместе:

* одна команда запускает весь процесс — парсинг + очистку,
* параметры (например, какая категория, сколько страниц) задаём через конфиг или `.env`,
* смоук-тесты проверяют, что данные корректны (есть все колонки, нет пустых URL, цены > 0).

👉 Цель: сделать проект удобным и повторяемым — так, чтобы его можно было запустить заново через неделю или дать другому человеку, и всё сработает.

---

## Фаза 0. Подготовка окружения

In [1]:
# TODO: Установите зависимости проекта через pip.
# Подсказка: используйте модуль запуска пакетов из Python или магию Jupyter (%pip).
# Обязательные пакеты: requests, beautifulsoup4, lxml, pandas, python-dotenv, tqdm (и при желании pyarrow).
# Пример шага: установить список пакетов и вывести подтверждение установки.

# Установка зависимостей
try:
    import sys, subprocess
    pkgs = ["requests", "beautifulsoup4", "lxml", "pandas", "python-dotenv", "tqdm", "pyarrow"]
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q"] + pkgs)
    print("✅ Зависимости установлены/обновлены")
except Exception as e:
    print("⚠️ Установите пакеты вручную:", e)

✅ Зависимости установлены/обновлены


In [2]:
# Хелперы для отображения и скачивания
import pandas as pd
from IPython.display import display
from pathlib import Path

IN_COLAB = False
try:
    import google.colab  # type: ignore
    from google.colab import data_table, files  # type: ignore
    data_table.enable_dataframe_formatter()
    IN_COLAB = True
except Exception:
    files = None  # type: ignore

def show_df(df: pd.DataFrame, title: str, n: int = 20):
    if title:
        print(title)
    display(df.head(n))

def show_csv(path: Path, title: str, n: int = 20):
    if not path.exists():
        print(f"Файл не найден: {path}")
        return
    try:
        df = pd.read_csv(path)
        show_df(df, title, n)
    except Exception as e:
        print("Не удалось прочитать CSV:", e)

def download_file(path: Path):
    """Скачивание файла в Colab. В Jupyter просто сообщает путь."""
    if not path.exists():
        print(f"Файл не найден: {path}")
        return
    if IN_COLAB and files is not None:
        files.download(str(path))
    else:
        print(f"Скачивание доступно в Colab. Путь к файлу: {path}")

In [3]:
# TODO: Создайте структуру папок проекта: data/raw, data/processed, logs.
# Подсказка: используйте pathlib.Path и метод mkdir(..., parents=True, exist_ok=True).
# Проверьте, что директории появились, выведите их абсолютные пути.
from pathlib import Path

BASE_DIR = Path.cwd()
RAW_DIR = BASE_DIR / 'data' / 'raw'
PROCESSED_DIR = BASE_DIR / 'data' / 'processed'
LOG_DIR = BASE_DIR / 'logs'

for path in [RAW_DIR, PROCESSED_DIR, LOG_DIR]:
  path.mkdir(parents = True, exist_ok = True)

print(f'Base dir: {BASE_DIR}\nRaw dir: {RAW_DIR}\nPocessed dir: {PROCESSED_DIR}\nLogs dir: {LOG_DIR}')

Base dir: d:\AIAcademy\Parser_1_project
Raw dir: d:\AIAcademy\Parser_1_project\data\raw
Pocessed dir: d:\AIAcademy\Parser_1_project\data\processed
Logs dir: d:\AIAcademy\Parser_1_project\logs


### Утилиты: логирование

In [4]:
# TODO: Реализуйте функцию get_logger(name) для логирования в консоль и файл.
# Требования:
# - уровень INFO;
# - 2 хэндлера: StreamHandler (консоль) и FileHandler (в logs/<name>.log);
# - формат логов: "%(asctime)s - %(levelname)s - %(name)s - %(message)s".
# Подсказка: не дублируйте хэндлеры при повторных запусках в Jupyter.


import logging

def get_logger(name: str, log_dir: Path = LOG_DIR) -> logging.Logger:
    log_dir.mkdir(parents = True, exist_ok = True)
    logger = logging.getLogger(name)
    logger.setLevel(logging.INFO)
    if logger.handlers:
        return logger
    ch = logging.StreamHandler(); ch.setLevel(logging.INFO)
    fh = logging.FileHandler(log_dir / f"{name}.log", encoding="utf-8"); fh.setLevel(logging.INFO)
    fmt = logging.Formatter("%(asctime)s - %(levelname)s - %(name)s - %(message)s")
    ch.setFormatter(fmt); fh.setFormatter(fmt)
    logger.addHandler(ch); logger.addHandler(fh)
    return logger


## Фаза 1. Мини‑парсер — Quotes to Scrape

In [5]:

# TODO: Импортируйте необходимые модули для сетевых запросов и HTML‑парсинга.
# Рекомендуемые инструменты: requests, bs4.BeautifulSoup, time.
# Создайте константы для базового URL и пути для CSV.
import csv, time, requests, re
from bs4 import BeautifulSoup

QUOTES_BASE_URL = "https://quotes.toscrape.com"
QUOTES_OUT = RAW_DIR / "quotes.csv"
quotes_logger = get_logger("quotes")
HEADERS = {"User-Agent": "Mozilla/5.0"}

In [6]:

# TODO: Реализуйте функцию fetch(url) с повторами и таймаутом.
# Логика:
# 1) выполнить GET с заголовком User-Agent;
# 2) при ошибке логировать предупреждение и делать экспоненциальную паузу;
# 3) после N попыток выбросить исключение.

def fetch(url: str, retries: int = 3, timeout: int = 15) -> str:
    for attempt in range(1, retries+1):
        try:
            r = requests.get(url, headers=HEADERS, timeout=timeout)
            r.raise_for_status()
            return r.text
        except Exception as e:
            quotes_logger.warning(f"Attempt {attempt}: {url} -> {e}")
            time.sleep(1.5 * attempt)
    raise RuntimeError(f"Failed to fetch {url} after {retries} attempts")

In [7]:

# TODO: Реализуйте функцию parse_quotes_page(html), которая:
# - находит блоки цитат;
# - извлекает поля: text, author, tags (теги объединить символом '|');
# - находит ссылку на следующую страницу (или None).
# Верните кортеж (список_словарей, next_url).

def parse_quotes_page(html: str):
    soup = BeautifulSoup(html, "lxml")
    quotes = []
    for q in soup.select(".quote"):
        text = q.select_one(".text").get_text(strip=True)
        author = q.select_one(".author").get_text(strip=True)
        tags = [t.get_text(strip=True) for t in q.select(".tags a.tag")]
        quotes.append({"text": text, "author": author, "tags": "|".join(tags)})
    next_link = soup.select_one("li.next a")
    next_url = QUOTES_BASE_URL + next_link["href"] if next_link else None
    return quotes, next_url

In [8]:

# TODO: Реализуйте функцию save_csv(rows, path, fieldnames, write_header=False).
# Требования:
# - создать директорию при необходимости;
# - использовать csv.DictWriter;
# - корректная работа с UTF-8 и переносами строк.

def save_csv(rows, path: Path, fieldnames, write_header: bool = False):
    path.parent.mkdir(parents=True, exist_ok=True)
    with path.open("a", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        if write_header:
            writer.writeheader()
        writer.writerows(rows)

In [9]:

# TODO: Реализуйте функцию run_quotes_scraper(), которая:
# - инициализирует логгер;
# - удаляет существующий CSV (если он есть);
# - итерируется по страницам, пока есть next_url;
# - сохраняет данные постранично;
# - возвращает общее количество собранных цитат.
# В конце выведите краткую статистику.

def run_quotes_scraper(show: bool = True, preview_rows: int = 15, download: bool = False):
    quotes_logger.info("Start scraping quotes")
    if QUOTES_OUT.exists():
        QUOTES_OUT.unlink()
    page_url = QUOTES_BASE_URL
    first = True; total = 0
    while page_url:
        html = fetch(page_url)
        rows, page_url = parse_quotes_page(html)
        save_csv(rows, QUOTES_OUT, ["text","author","tags"], write_header=first)
        first = False; total += len(rows)
        quotes_logger.info(f"Saved {len(rows)} quotes. Next: {page_url}")
        time.sleep(1.0)
    quotes_logger.info(f"Done. Total quotes: {total}")
    if show:
        show_csv(QUOTES_OUT, title=f"Quotes — предпросмотр ({preview_rows})", n=preview_rows)
    if download:
        download_file(QUOTES_OUT)
    return total

run_quotes_scraper()

2025-10-10 18:59:57,560 - INFO - quotes - Start scraping quotes
2025-10-10 18:59:58,924 - INFO - quotes - Saved 10 quotes. Next: https://quotes.toscrape.com/page/2/
2025-10-10 19:00:01,023 - INFO - quotes - Saved 10 quotes. Next: https://quotes.toscrape.com/page/3/
2025-10-10 19:00:02,940 - INFO - quotes - Saved 10 quotes. Next: https://quotes.toscrape.com/page/4/
2025-10-10 19:00:04,939 - INFO - quotes - Saved 10 quotes. Next: https://quotes.toscrape.com/page/5/
2025-10-10 19:00:06,814 - INFO - quotes - Saved 10 quotes. Next: https://quotes.toscrape.com/page/6/
2025-10-10 19:00:08,683 - INFO - quotes - Saved 10 quotes. Next: https://quotes.toscrape.com/page/7/
2025-10-10 19:00:10,676 - INFO - quotes - Saved 10 quotes. Next: https://quotes.toscrape.com/page/8/
2025-10-10 19:00:12,570 - INFO - quotes - Saved 10 quotes. Next: https://quotes.toscrape.com/page/9/
2025-10-10 19:00:14,473 - INFO - quotes - Saved 10 quotes. Next: https://quotes.toscrape.com/page/10/
2025-10-10 19:00:16,406 - 

Quotes — предпросмотр (15)


,text,author,tags
0,“The world as we have created it is a process ...,Albert Einstein,change|deep-thoughts|thinking|world
1,"“It is our choices, Harry, that show what we t...",J.K. Rowling,abilities|choices
2,“There are only two ways to live your life. On...,Albert Einstein,inspirational|life|live|miracle|miracles
3,"“The person, be it gentleman or lady, who has ...",Jane Austen,aliteracy|books|classic|humor
4,"“Imperfection is beauty, madness is genius and...",Marilyn Monroe,be-yourself|inspirational
5,“Try not to become a man of success. Rather be...,Albert Einstein,adulthood|success|value
6,“It is better to be hated for what you are tha...,André Gide,life|love
7,"“I have not failed. I've just found 10,000 way...",Thomas A. Edison,edison|failure|inspirational|paraphrased
8,“A woman is like a tea bag; you never know how...,Eleanor Roosevelt,misattributed-eleanor-roosevelt
9,"“A day without sunshine is like, you know, nig...",Steve Martin,humor|obvious|simile


100

## Фаза 2. Маркетплейс — двухшаговый парсинг

In [10]:
# TODO: Подготовьте импорты и переменные для сайта:
# - BASE_URL;
# - MARKET_OUT (путь к RAW CSV);
# - логгер;
# Подсказка: используйте те же утилиты логирования.

import csv, time, requests, re
from bs4 import BeautifulSoup
from pathlib import Path


BASE_URL = 'https://liquipedia.net/'
OUT_DIR = Path.cwd() / 'data' / 'raw' / 'games.csv'
HEADERS = {'User-Agent': 'Mozilla/5.0'}
games_logger = get_logger('game_tournaments')

In [11]:
# TODO: Реализуйте функцию fetch_site(url) по аналогии с fetch(url), но с другими таймаутами/повторами.
# Требования такие же: заголовок User-Agent, обработка исключений, паузы между попытками.

def fetch_site(url: str, retries: int = 5, timeout: int = 20) -> str:
    for attempt in range(1, retries + 1):
        try:
            req = requests.get(url, HEADERS, timeout = timeout)
            req.raise_for_status()
            return req.text
        except Exception as e:
            games_logger.warning(f'Attempt {attempt}: {url} -> {e}')
            time.sleep(timeout / 5)
    raise RuntimeError(f'Failed to fetch {url} after {retries} attempts')

In [20]:
# TODO: Реализуйте функцию get_item_links_from_category(html):
# - распарсьте HTML категории;
# - соберите все <a href="...">, нормализуйте в абсолютные ссылки;

def get_game_links(html: str) -> list:
    soup = BeautifulSoup(html, 'lxml')
    game_urls = []
    for game in soup.select('.wiki-card__link'):
        if game.get('href') is not None and game.get_text() != 'Hub' and game.get_text() != 'Lab':
            game_url = BASE_URL + game.get('href')
        if game_url not in game_urls:
            game_urls.append(game_url)
    return game_urls

In [ ]:
def add_tournament(list_to_add: list, element_to_add: str, tourn_limit: int = None) -> None:
    if len(list_to_add) < tourn_limit or tourn_limit == None:
        list_to_add.append(element_to_add)

def clean_data(info: dict) -> dict:
    result_dict = {}
    result_dict.update(info)
    result_dict['Upcoming'] = ''
    result_dict['Ongoing'] = ''
    result_dict['Completed'] = ''
    
    for key in info.keys():
        if key != 'Game_name':
            for tourn_name in info[key]:
                result_dict[key] = result_dict[key] + tourn_name + ', '
    return result_dict

def parse_game_page(game_link: str, tournaments_limit: int = 5) -> dict:
    game_info = {
        'Game_name' : None,
        'Upcoming' : [],
        'Ongoing' : [],
        'Completed' : []
    }
    
    game_html = fetch_site(game_link, 2, 60)
    soup = BeautifulSoup(game_html, 'lxml')
    game_name = soup.select_one('.brand-subtitle').get_text()
    game_info['Game_name'] = game_name
    tournaments_names = soup.select_one('.tournaments-list')
    for tn_child in tournaments_names.children:
        tournaments_html = tn_child.select('.tournament-name') if tn_child.select('.tournament-name') else tn_child.select('.tournaments-list-name')
        for tourn in tournaments_html:
            if tn_child.select_one('.tournaments-list-heading').get_text() == 'Upcoming':
                add_tournament(game_info['Upcoming'], tourn.get_text(), tournaments_limit)
            elif tn_child.select_one('.tournaments-list-heading').get_text() == 'Ongoing':
                add_tournament(game_info['Ongoing'], tourn.get_text(), tournaments_limit)
            elif tn_child.select_one('.tournaments-list-heading').get_text() == 'Completed' or 'Concluded':
                add_tournament(game_info['Completed'], tourn.get_text(), tournaments_limit)
    return game_info

{'Game_name': 'Dota 2', 'Upcoming': 'Moscow Championship 2025, APPL Malaysia, APPL Indonesia, BLAST Slam V CN CQ, CCT S2 SA Series 1, ', 'Ongoing': 'Kobold League II: Sweat Division, Impacto Solar 4, Lunar Snake 3, ACS 2, LFL Streamer Mix, ', 'Completed': 'RES Unchained 2 BLAST EU OQ2, RES Unchained 2 BLAST SEA OQ2, RES Unchained 2 BLAST EU OQ1, RES Unchained 2 BLAST SEA OQ1, PGL Wallachia S6 CN CQ, '}


In [48]:
# TODO: Реализуйте функцию run_site_scraper(...), которая:
# - принимает category, pages_limit, per_page_limit, задержки;
# - собирает ссылки объявлений с каждой страницы категории;
# - для каждой ссылки загружает карточку и извлекает title/price;
# - постранично сохраняет результат в CSV;
# - возвращает общее количество валидных объявлений.
# Выведите статистику по страницам и итог.

def run_site_scrapper(tournaments_limit: int = None, retries: int = 1, timeout: int = 20):
    site_html = fetch_site(BASE_URL, retries, timeout)
    list_of_game_pages = get_game_links(site_html)
    print(list_of_game_pages)
    total = 0
    rows = []
    first_time = True
    for game_page in list_of_game_pages:
        rows = [clean_data(parse_game_page(game_page, tournaments_limit))]
        save_csv(rows, OUT_DIR, ['Game_name', 'Upcoming', 'Ongoing', 'Completed'], first_time)
        first_time = False
        total = total + 1
        games_logger.info(f'Saved {rows[len(rows) - 1]["Game_name"]} info.')
        time.sleep(timeout / 2)
    games_logger.info(f'Done. Total games: {total}')
    show_csv(OUT_DIR, title = f'Games info', n = 10)
    download_file(OUT_DIR)
    return total

run_site_scrapper(5, 1, 20)

['https://liquipedia.net//dota2/Main_Page', 'https://liquipedia.net//counterstrike/Main_Page', 'https://liquipedia.net//valorant/Main_Page', 'https://liquipedia.net//mobilelegends/Main_Page', 'https://liquipedia.net//leagueoflegends/Main_Page', 'https://liquipedia.net//rocketleague/Main_Page', 'https://liquipedia.net//apexlegends/Main_Page', 'https://liquipedia.net//overwatch/Main_Page', 'https://liquipedia.net//pubgmobile/Main_Page', 'https://liquipedia.net//rainbowsix/Main_Page', 'https://liquipedia.net//fighters/Main_Page', 'https://liquipedia.net//starcraft2/Main_Page', 'https://liquipedia.net//pubg/Main_Page', 'https://liquipedia.net//ageofempires/Main_Page', 'https://liquipedia.net//honorofkings/Main_Page', 'https://liquipedia.net//smash/Main_Page', 'https://liquipedia.net//callofduty/Main_Page', 'https://liquipedia.net//brawlstars/Main_Page', 'https://liquipedia.net//warcraft/Main_Page', 'https://liquipedia.net//worldoftanks/Main_Page', 'https://liquipedia.net//easportsfc/Main_P

2025-10-10 20:07:59,803 - INFO - game_tournaments - Saved Dota 2 info.
2025-10-10 20:08:10,770 - INFO - game_tournaments - Saved Counter-Strike info.
2025-10-10 20:08:21,474 - INFO - game_tournaments - Saved VALORANT info.
2025-10-10 20:08:32,105 - INFO - game_tournaments - Saved Mobile Legends info.
2025-10-10 20:08:42,930 - INFO - game_tournaments - Saved League of Legends info.
2025-10-10 20:08:53,612 - INFO - game_tournaments - Saved Rocket League info.
2025-10-10 20:09:04,291 - INFO - game_tournaments - Saved Apex Legends info.
2025-10-10 20:09:14,970 - INFO - game_tournaments - Saved Overwatch info.
2025-10-10 20:09:26,089 - INFO - game_tournaments - Saved PUBG Mobile info.
2025-10-10 20:09:36,786 - INFO - game_tournaments - Saved Rainbow Six info.
2025-10-10 20:09:48,281 - INFO - game_tournaments - Saved Fighting Games info.
2025-10-10 20:09:58,900 - INFO - game_tournaments - Saved StarCraft II info.
2025-10-10 20:10:09,479 - INFO - game_tournaments - Saved PUBG info.
2025-10-10

Games info


,Game_name,Upcoming,Ongoing,Completed
0,Dota 2,"Moscow Championship 2025, APPL Malaysia, APPL ...","Kobold League II: Sweat Division, Impacto Sola...","RES Unchained 2 BLAST EU OQ2, RES Unchained 2 ..."
1,Counter-Strike,"eXTREMESLAND 2025, ESL Impact League Season 8,...","Virslīga Season 5, Esporto A Lyga Season 2, Me...","Female Pro League S1 OQ #1, Winline Insight Se..."
2,VALORANT,"Monsters: Turkish Qual., Predator League Singa...","Denmark League Autumn, National Women's League...","PV Pro Series Weekly #7, PV Weekly #32, Celeri..."
3,Mobile Legends,"IESF WEC 2025, Light of Dawn - S3, LEN - Liga ...","MPL KH Season 9, MLBB Super League S2, MCC S6,...","MEC '25 Fall - TD, MEC '25 Fall - R4, IESF '25..."
4,League of Legends,"CBLOL Split 1, Circuito Desafiante Split 1, LC...","LTK Trials of Twilight, Svenska Onlineligan Au...","LTA South 2026 Promotion, NLC Regional Promoti..."
5,Rocket League,"FeWC 25 - Africa Regionals, FeWC 25 - Europe R...","NRLS - Season 2, Ontario Campus Carball 2025-2...","El Potrero #1, CRL Fall 25 - Open1, Double Com..."
6,Apex Legends,"ALGS: LCQ - APAC-N, ALGS: LCQ - APAC-S, ALGS: ...","ALGS: APAC North PL Split 2 2025, ALGS: Americ...","ALGS: S2 CC #4 - APAC-N, ALGS: S2 CC #4 - Amer..."
7,Overwatch,"OWCS World Finals, FACEIT League: Masters Show...","Homecoming Fall 2025 Open, Homecoming Fall 202...","Nittany Invitational Playoffs, SwitzerLAN 2025..."
8,PUBG Mobile,"SEL 2025 Champ., PMCC NA Fall 2025, BMIC 2025,...","UPG MOBILE CUP, PEN 2025, PMSL MENA Fall 2025,...","Japan League S5: P2, PMCL SEA Fall 2025, PMNC ..."
9,Rainbow Six,"BLAST R6 Major Munich 2025, Axis Series - Seas...","ELC Fall '25 - Championship, CCL Season 3, Tal...","OCS S7 Contenders, Nittany Invitational, Talen..."


Скачивание доступно в Colab. Путь к файлу: d:\AIAcademy\Parser_1_project\data\raw\games.csv


49

## Фаза 3. Очистка и валидация

In [ ]:

# TODO: Импортируйте pandas и другие нужные модули (datetime, re).
# Объявите пути: RAW_CSV (в data/raw), CLEAN_CSV и CLEAN_PQ (в data/processed).

In [ ]:

# TODO: Напишите функцию normalize_title(s), которая:
# - обрезает пробелы по краям;
# - заменяет множественные пробелы на один.
# Верните нормализованную строку.


In [ ]:

# TODO: Реализуйте функцию clean_market_data(...):
# - загрузите RAW CSV;
# - проверьте наличие обязательных столбцов (title, price, url);
# - удалите дубликаты по url и пустые url;
# - приведите price к числу (coerce), оставьте только > 0 и разумный верхний порог;
# - добавьте служебные поля: source, category, currency, date_scraped (ISO‑8601 UTC);
# - сохраните CSV и Parquet (если установлен соответствующий движок);
# - верните количество валидных строк.
# Выведите предпросмотр очищенных данных.


## Фаза 4. Воспроизводимость: пайплайн и смоук‑тесты

In [ ]:

# TODO: Реализуйте функцию run_pipeline(), которая:
# - запускает мини‑парсер (опционально);
# - запускает маркет‑парсер;
# - запускает очистку;
# - печатает сводку scraped/cleaned и возвращает кортеж.


In [ ]:

# TODO: Реализуйте функцию smoke_tests(), которая проверяет базовые инварианты:
# - наличие RAW и CLEAN файлов;
# - clean не длиннее raw;
# - наличие столбцов в clean: title, price, currency, url, source, category, date_scraped;
# - все цены в clean > 0;
# - нет пустых url.
# В конце выведите 'SMOKE CHECKS OK' при успехе.



### Этика и право
- Соблюдайте robots.txt и условия сайта.
- Держите паузы между запросами, не создавайте лишней нагрузки.
- Не собирайте персональные данные, не обходите защиту.

### DoD — критерии приёмки
- Мини‑парсер собрал ≥ 100 цитат;
- Маркет‑парсер собрал ≥ 200 объявлений при разумных лимитах;
- Очищенный CSV/Parquet содержит нужные столбцы;
- Смоук‑тесты проходят без ошибок;
- Весь конвейер запускается одной функцией.
